In [ ]:
#Questions
    #What to do with unbalanced data
    #How to align test and train data from feature engineering

In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [46]:
from fastai.structured import *
from fastai.column_data import *
from sklearn.metrics import roc_auc_score

np.set_printoptions(threshold = 50, edgeitems = 20)

PATH = 'data/'

In [48]:
ls {PATH}

application_test.csv         HomeCredit_columns_description.csv
application_test.csv.zip     installments_payments.csv
application_train.csv        installments_payments.csv.zip
application_train.csv.zip    POS_CASH_balance.csv
bureau_balance.csv           POS_CASH_balance.csv.zip
bureau_balance.csv.zip       previous_application.csv
bureau.csv                   previous_application.csv.zip
bureau.csv.zip               sample_submission.csv
credit_card_balance.csv      sample_submission.csv.zip
credit_card_balance.csv.zip  subm/
df_pref                      tmp/


In [49]:
#data = pd.read_feather('tmp/data')
data = pd.read_csv(PATH + 'application_train.csv')
data.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
#data_test = pd.read_feather('tmp/data_test')
data_test = pd.read_csv(PATH + 'application_test.csv')
data_test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# nrow = 1000
#application_train = pd.read_csv(path + 'application_train.csv', nrow = 1000)
#bureau_balance = pd.read_csv(path + 'bureau_balance.csv')
#bureau = pd.read_csv(path + 'bureau.csv')
#credit_card_balance = pd.read_csv(path + 'credit_card_balance.csv')
#installments_payments = pd.read_csv(path + 'installments_payments.csv')
#pos_cash_balance = pd.read_csv(path+'POS_CASH_balance.csv')
#previous_application = pd.read_csv(path + 'previous_application.csv')

In [55]:
cat_vars = list(data.select_dtypes('object').columns); len(cat_vars)


16

In [59]:
cat_vars

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE']

In [54]:
contin_vars = list(data.select_dtypes(np.number).columns); len(contin_vars)

106

In [56]:
dep = 'TARGET'
data = data[cat_vars+contin_vars+[dep, 'SK_ID_CURR']].copy()

In [58]:
data_test[dep] = 0
data_test = data_test[cat_vars+contin_vars+[dep, 'SK_ID_CURR']].copy()

In [68]:
for v in cat_vars: data[v] = data[v].astype('category').cat.as_ordered(False)

AttributeError: 'DataFrame' object has no attribute 'category'

In [62]:
train_cats(data)

In [69]:
apply_cats(data_test, data)

AttributeError: 'DataFrame' object has no attribute 'dtype'

In [64]:
data.shape, data_test.shape

((307511, 124), (48744, 124))

# Deep Learning

Merics

In [7]:
def rmse(x,y):
    return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [roc_auc_score(y_train, m.predict(X_train)), roc_auc_score(y_valid, m.predict(X_valid)),
          m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m,'oob_score_'): res.append(m.oob_score_)
    print(res)

ModelData object

In [ ]:
df, y, nas, mapper = proc_df(joined, 'TARGET', do_scale=True)

In [ ]:
md = ColumnarModelDate.from_data_frame(PATH, val_idx, df, yl.astype(np.float32), 
                                       cat_flds = cat_vars, bs=128, test_df=df_test )